# 2.1
1個120円のりんごと1個150円のなしを合わせて10個買ったら、代金の合計が1440円でした。りんごとなしをそれぞれ何個買ったでしょうか。

In [4]:
import pulp

problem = pulp.LpProblem("SLE", pulp.LpMaximize)
# 第1引数は問題の名前。任意。SLE=Simultaneous Linear Equations
# 第2引数は最大化問題を解くことを指定しているが、今回は意味なし。

x = pulp.LpVariable("x", cat="Continuous")
y = pulp.LpVariable("y", cat="Continuous")

# 制約式
# += を使うこと、== を使うことに注意。
problem += 120 * x + 150 * y == 1440
problem += x + y == 10

status = problem.solve(pulp.PULP_CBC_CMD(msg=0))
# PULP_CBC_CMD(msg=0)でログメッセージを抑制

print("Status:", pulp.LpStatus[status])
print("x=", x.value(), "y=", y.value())

Status: Optimal
x= 2.0 y= 8.0


# 2.2
ある工場では製品pとqを製造しています。製品pとqを製造するには原料mとnが必要で、次のことが分かっています。

- 製品pを1kg製造するには原料mが1kg、原料nが2kg必要である。
- 製品qを1kg製造するには原料mが3kg、原料nが1kg必要である。
- 原料mの在庫は30kg、原料nの在庫は40kgである。
- 単位量当たりの利得は、製品pは1万円、製品nは2万円である。

この条件で利得を最大にするためには、製品pとqをそれぞれ何kg製造すればよいでしょうか。

製品pの製造量をxkg、製品qの製造量をykgとすると、
$$
x + 3y \le 30 \\
2x + y \le 40 \\
x \ge 0, y \ge 0 \\
$$
の条件のもとで、利得の合計$x+2y$を最大化すればよい。

In [3]:
import pulp

problem = pulp.LpProblem("LP", pulp.LpMaximize)

x = pulp.LpVariable("x", cat="Continuous")
y = pulp.LpVariable("y", cat="Continuous")

problem += 1 * x + 3 * y <= 30
problem += 2 * x + 1 * y <= 40
problem += x >= 0
problem += y >= 0
problem += x + 2 * y  # 制約式の形でないものは目的関数と見做される

status = problem.solve(pulp.PULP_CBC_CMD(msg=0))

print("Status:", pulp.LpStatus[status])
print("x=", x.value(), "y=", y.value(), "obj=", problem.objective.value())

Status: Optimal
x= 18.0 y= 4.0 obj= 26.0


# 2.3 規模の大きな問題
ある工場では製品p1、p2、p3、p4の4種類を製造しています。製品p1、p2、p3、p4の製造には、原料m1、m2、m3の３種類が必要です。
また、その量はファイルrequires.csvに記述されています。
|p|m|require|
|:---:|:---:|:---:|
|p1|m1|2|
|p1|m2|0|
|p1|m3|1|
|p2|m1|3|
|・・・|・・・|・・・|

本データの2～4行目は、製品p1を1kg製造するには原料m1が2kg、原料m2が0kg、原料m3が1kg必要であることを表しています。
また、原料m1、m2、m3の在庫は、stocks.csvに記述されています。
|m|stock|
|:---:|:---:|
|m1|35|
|m2|22|
|m3|27|

最後に、製品p1、p2、p3、p4を生産した際の利得はgains.csvに記述されています。
|p|gain|
|:---:|:---:|
|p1|3|
|p2|4|
|p3|4|
|p4|5|

このとき、利得を最大にするためには、製品p1、p2、p3、p4をそれぞれ何kg製造すればよいでしょうか。

In [7]:
import pandas as pd
import pulp

stock_df = pd.read_csv("data/stocks.csv")
require_df = pd.read_csv("data/requires.csv")
gain_df = pd.read_csv("data/gains.csv")

# リストの定義
P = gain_df["p"].tolist()
# ['p1', 'p2', 'p3', 'p4']
M = stock_df["m"].tolist()
# ['m1', 'm2', 'm3']

# 定数の定義（辞書化）
stock = stock_df.set_index("m").to_dict()["stock"]
# {'m1': 35, 'm2': 22, 'm3': 27}
require = require_df.set_index(["p", "m"]).to_dict()["require"]
# {('p1', 'm1'): 2, ('p1', 'm2'): 0, ('p1', 'm3'): 1, ('p2', 'm1'): 3,
#  ('p2', 'm2'): 2, ('p2', 'm3'): 0, ('p3', 'm1'): 0, ('p3', 'm2'): 2,
#  ('p3', 'm3'): 2, ('p4', 'm1'): 2, ('p4', 'm2'): 2, ('p4', 'm3'): 2}
gain = gain_df.set_index("p").to_dict()["gain"]
# {'p1': 3, 'p2': 4, 'p3': 4, 'p4': 5}

# 線形計画問題の定義
problem = pulp.LpProblem("LP2", pulp.LpMaximize)

# 変数の定義（まとめて定義）
x = pulp.LpVariable.dicts("x", P, cat="Continuous")
# {'p1': x_p1, 'p2': x_p2, 'p3': x_p3, 'p4': x_p4}

# 制約式の定義
for p in P:
    problem += x[p] >= 0
for m in M:
    problem += pulp.lpSum([require[p, m] * x[p] for p in P]) <= stock[m]

# 目的関数の定義
problem += pulp.lpSum([gain[p] * x[p] for p in P])

status = problem.solve(pulp.PULP_CBC_CMD(msg=0))
print("Status:", pulp.LpStatus[status])

for p in P:
    print(p, x[p].value())

print("obj=", problem.objective.value())

Status: Optimal
p1 12.142857
p2 3.5714286
p3 7.4285714
p4 0.0
obj= 80.42857099999999


### 生産量をkgではなく個数として、整数計画問題を解いてみる

In [8]:
import pandas as pd
import pulp

stock_df = pd.read_csv("data/stocks.csv")
require_df = pd.read_csv("data/requires.csv")
gain_df = pd.read_csv("data/gains.csv")

# リストの定義
P = gain_df["p"].tolist()
# ['p1', 'p2', 'p3', 'p4']
M = stock_df["m"].tolist()
# ['m1', 'm2', 'm3']

# 定数の定義（辞書化）
stock = stock_df.set_index("m").to_dict()["stock"]
# {'m1': 35, 'm2': 22, 'm3': 27}
require = require_df.set_index(["p", "m"]).to_dict()["require"]
# {('p1', 'm1'): 2, ('p1', 'm2'): 0, ('p1', 'm3'): 1, ('p2', 'm1'): 3,
#  ('p2', 'm2'): 2, ('p2', 'm3'): 0, ('p3', 'm1'): 0, ('p3', 'm2'): 2,
#  ('p3', 'm3'): 2, ('p4', 'm1'): 2, ('p4', 'm2'): 2, ('p4', 'm3'): 2}
gain = gain_df.set_index("p").to_dict()["gain"]
# {'p1': 3, 'p2': 4, 'p3': 4, 'p4': 5}

# 整数計画問題の定義
problem = pulp.LpProblem("IP", pulp.LpMaximize)

# 変数の定義（まとめて定義）
x = pulp.LpVariable.dicts("x", P, cat="Integer")
# {'p1': x_p1, 'p2': x_p2, 'p3': x_p3, 'p4': x_p4}

# 制約式の定義
for p in P:
    problem += x[p] >= 0
for m in M:
    problem += pulp.lpSum([require[p, m] * x[p] for p in P]) <= stock[m]

# 目的関数の定義
problem += pulp.lpSum([gain[p] * x[p] for p in P])

status = problem.solve(pulp.PULP_CBC_CMD(msg=0))
print("Status:", pulp.LpStatus[status])

for p in P:
    print(p, x[p].value())

print("obj=", problem.objective.value())

Status: Optimal
p1 13.0
p2 3.0
p3 7.0
p4 -0.0
obj= 79.0
